This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

K_pU8sYy6BXHq-YYhCsM


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2018-01-01&order=desc&api_key=K_pU8sYy6BXHq-YYhCsM'
r = requests.get(url)
json_data = r.json()

# Data sample for a day
print(json_data['dataset']['data'][0])

['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None]


In [4]:
# Dataset for AFX_X
json_dumps = json.dumps(json_data, indent=2)

In [5]:
# organize access to price data from dataset

afx_x_2017 = {
    'raw_data': json_data,
    'columns': json_data['dataset']['column_names'][:],
    'data': json_data['dataset']['data'][:]
}


In [6]:
# list of the names of the pertinent columns
afx_x_col = afx_x_2017['columns']

# list of the lists of price data
afx_x_data = afx_x_2017['data']

In [7]:
# Empty dictionary to insert 2017 data for AFX_X by date
afx_x_dict = {}

# Fill in the empty dictionary above
for row in afx_x_data:
    data_row_dict = dict(zip(afx_x_col, row))
    afx_x_dict[data_row_dict['Date']] = data_row_dict

In [8]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [9]:
# Calculating the highest and lowest opening prices for 2017

# Dictionary comprehension of opening prices by date
opening_prices = {k:v['Open'] for k, v in afx_x_dict.items() if v['Open'] != None}

# max opening value
afx_x_max_ov = max(opening_prices.values())

# key of max opening value
afx_x_max_ok = max(opening_prices, key= lambda key: opening_prices[key])

# min opening value
afx_x_min_ov = min(opening_prices.values())

# key of min opening value
afx_x_min_ok = min(opening_prices, key= lambda key: opening_prices[key])


print(f'The highest opening price for 2017 was on', afx_x_max_ok, 'at', afx_x_max_ov)
print(f'The lowest opening price for 2017 was on', afx_x_min_ok, 'at', afx_x_min_ov)

The highest opening price for 2017 was on 2017-12-14 at 53.11
The lowest opening price for 2017 was on 2017-01-24 at 34.0


In [10]:
# Largest change in any one day based on high and low price

# library of change in share price by day in 2017
day_change = {k:abs(v['High'] - v['Low']) for k, v in afx_x_dict.items()}

# max change in one day
afx_x_max_ch = max(day_change.values())

# key of max change in one day
afx_x_max_ch_k = max(day_change, key= lambda key: day_change[key])


print(f'The highest one day change in price was on', afx_x_max_ch_k, 'for', round(afx_x_max_ch, 2))

The highest one day change in price was on 2017-05-11 for 2.81


In [11]:
# Largest change between any two days based on closing price


# List of dates and closing prices for those dates in 2017
dc = list((k,v['Close']) for k, v in afx_x_dict.items())

# Empty list to hold tuples with values for change in closing price between 2 days
two_day_change = []

# Loop to make a tuple with the dates and change in closing price between those dates to add to above list
for i in range(len(dc) - 1):
    two_dc = (dc[i][0]+' & '+dc[i+1][0], abs(dc[i][1] - dc[i+1][1]))
    two_day_change.append(two_dc)

# The 2 days with the largest change in closing price    
largest_2_dc = max(two_day_change, key=lambda item:item[1])

print(f'The largest change in closing price between any two days during 2017 was', round(largest_2_dc[1], 2), 'between', largest_2_dc[0])

The largest change in closing price between any two days during 2017 was 2.56 between 2017-08-09 & 2017-08-08


In [12]:
# Average daily trading volume during 2017

# dictionary of trading volume by day for 2017
traded_volume = {k:v['Traded Volume'] for k, v in afx_x_dict.items() if v['Traded Volume'] != None}

# average daily trading volume for 2017
avg_daily_vol = sum(traded_volume.values()) / len(traded_volume.values())


print(f'The daily trading volume during 2017 was', round(avg_daily_vol, 2), 'shares')

The daily trading volume during 2017 was 89124.34 shares


In [13]:
# Median trading volume during 2017

# import median function from statistics module.
from statistics import median

# median volume for 2017
median_vol = median(traded_volume.values())


print(f'The median trading volume during 2017 was', median_vol, 'shares')

The median trading volume during 2017 was 76286.0 shares
